In [60]:
!pip install backtesting
!pip install ta-lib
import numpy as np
import pandas as pd
import requests
import shioaji as sj
import datetime
from datetime import date, timedelta
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from backtesting import Backtest, Strategy 
from backtesting.lib import crossover
from backtesting.test import SMA 
import talib
import os
from dotenv import load_dotenv
# 加載 .env 文件中的環境變數
load_dotenv()

# 建立API物件，simulation=True是代表測試帳號
api = sj.Shioaji(simulation=True)

shioaji_secret = os.getenv("SHIOAJI_SECRETKEY", None)
shioaji_apikey = os.getenv("SHIOAJI_APIKEY", None)


# 登入你的key
# accounts = api.login("YOUR_API_KEY", "YOUR_SECRET_KEY")
accounts = api.login(shioaji_apikey, shioaji_secret)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 786.7 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173925 sha256=560877e47459641ebddce6c01127f5f5d381980356812aa8abfb70d907eaad5b
  Stored in directory: /Users/harry/Library/Caches/pip/wheels/2c/56/19/bf7ee5e164aa99a976e3f64841c83b5ae0391c59d9aec011d0
Successfully built backtesting
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 368.5/368.5 kB 1.1 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for ta-lib: filename=TA_Lib-0.4.32-cp311-cp311-macosx_11_0_arm64.whl size=394794 sha256=d1b977b79c21f1a2e84adde5044639e64aa211cad8690e9d730d4e96be73c35a
  Stored in directory: /Users/harry/Library/Caches/pip/wheels/df/89/6e/ecca68b9423ad205f55546bf926be3e13a26d055c3adf0ff0f
Successfully built ta-lib


/opt/homebrew/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

/opt/homebrew/anaconda3/lib/python3.11/site-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),
/opt/homebrew/anaconda3/lib/python3.11/site-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),


Response Code: 0 | Event Code: 0 | Info: host '210.59.255.161:80', hostname '210.59.255.161:80' IP 210.59.255.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up


In [61]:
#====定義日期 ex:1年,2年,3年======
today = date.today()
oneYear= 365
twoYear = 730
delta_oneYear = timedelta(days=oneYear)
delta_twoYear = timedelta(days=twoYear)
date_oneYear_ago = today - delta_oneYear
date_twoYear_age = today - delta_twoYear
testDate=today-timedelta(days=2)
lastWeek=today-timedelta(days=9)

# ==== end
# ====篩選週漲幅20% & 成值前100名 ====
##成值前100
scanners = api.scanners(
    scanner_type=sj.constant.ScannerType.AmountRank,
    count=100,
    date='2024-07-05'
)
scanners
amout_df = pd.DataFrame(s.__dict__ for s in scanners)
amout_df.ts = pd.to_datetime(amout_df.ts)
amout_df
stock_code_array = amout_df['code'].values

##成值前100 & 週漲幅20%(含以上）
amount_raise_stock = []
def raise_twenty_percent(stock_code):
    kbars = api.kbars(
        contract=api.Contracts.Stocks[stock_code], 
        start=str(lastWeek), 
        end=str(today), 
    )
    df = pd.DataFrame({**kbars})
    df.ts = pd.to_datetime(df.ts)
    df.set_index('ts',inplace=True)
    
    # 取週用resample('W') ,週k最後一天13:30的K棒 , dropna清除NaN的值
    weekly_df=df.resample('W').last().dropna()
    last_week_close = weekly_df['Close'].iloc[-2]
    this_week_close = weekly_df['Close'].iloc[-1]
    if this_week_close >= last_week_close*1.2:
        amount_raise_stock.append(stock_code)
    

for stock_code in stock_code_array:
    raise_twenty_percent(stock_code)
print(amount_raise_stock)
# ====end


['8028', '2464', '3583', '6187', '2467', '2365', '5443', '3048', '3047', '6442', '2543', '8064', '4562']


In [62]:
# 取得週k線
def weekly_five_average_strategy(stock_code,start_date,end_date)
kbars = api.kbars(
    contract=api.Contracts.Stocks["6442"], 
    start="2023-01-01", 
    end="2024-07-05", 
)
df = pd.DataFrame({**kbars})
df.ts = pd.to_datetime(df.ts)
df.set_index('ts',inplace=True)


# 取週用resample('W') ,週k最後一天13:30的K棒 , dropna清除NaN的值
weekly_df=df.resample('W').last().dropna()
print(weekly_df)
print(weekly_df['Close'][1])
# 計算5日均線和30日均線
weekly_df['5MA'] = weekly_df['Close'].rolling(window=5).mean()
weekly_df['30MA'] = weekly_df['Close'].rolling(window=30).mean()

# 繪製收盤價、5日均線和30日均線
plt.figure(figsize=(20, 6))
plt.plot(weekly_df.index, weekly_df['Close'], label='Close Price', color='blue')
plt.plot(weekly_df.index, weekly_df['5MA'], label='5-day MA', color='red')
plt.plot(weekly_df.index, weekly_df['30MA'], label='30-day MA', color='green')
plt.xlabel('Date')
plt.ylabel('Price')
plt.title('Weekly Stock Price with Moving Averages')
plt.legend()
plt.grid(True)
plt.show()

SyntaxError: expected ':' (3280659665.py, line 2)

SDK NOTICE Sun Jul 07 22:37:51.700 2024 solClient.c:11120                    (17357b000) Session '(c0,s1)_sinopac' keep-alive on tcp_TxRx detected session down, client name 'PYAPI/F129970274/0707/070855/456352/118.150.47.40', VPN name 'sinopac', peer host '210.59.255.161:80' address 'IP 210.59.255.161', connection 'tcp_TxRx' local address 'IP 192.168.100.2:56463'


Response Code: 0 | Event Code: 12 | Info: Session '(c0,s1)_sinopac' keep-alive on tcp_TxRx detected session down, client name 'PYAPI/F129970274/0707/070855/456352/118.150.47.40', VPN name 'sinopac', peer host '210.59.255.161:80' address 'IP 210.59.255.161', connection 'tcp_TxRx' local address 'IP | Event: Session reconnecting
Response Code: 0 | Event Code: 13 | Info: host '210.59.255.161:80', hostname '210.59.255.161:80' IP 210.59.255.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total reconnection attempt 1 of 10) | Event: Session reconnected


SDK NOTICE Sun Jul 07 22:37:52.189 2024 solClient.c:11120                    (31351f000) Session '(c2,s1)_sinopac' keep-alive on tcp_TxRx detected session down, client name 'PYAPI/F129970274/0707/134447/567859/118.150.47.40', VPN name 'sinopac', peer host '210.59.255.161:80' address 'IP 210.59.255.161', connection 'tcp_TxRx' local address 'IP 192.168.100.2:57912'


Response Code: 0 | Event Code: 12 | Info: Session '(c2,s1)_sinopac' keep-alive on tcp_TxRx detected session down, client name 'PYAPI/F129970274/0707/134447/567859/118.150.47.40', VPN name 'sinopac', peer host '210.59.255.161:80' address 'IP 210.59.255.161', connection 'tcp_TxRx' local address 'IP | Event: Session reconnecting
Response Code: 0 | Event Code: 13 | Info: host '210.59.255.161:80', hostname '210.59.255.161:80' IP 210.59.255.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total reconnection attempt 1 of 10) | Event: Session reconnected


...

Response Code: 0 | Event Code: 12 | Info: Session '(c0,s1)_sinopac' keep-alive on tcp_TxRx detected session down, client name 'PYAPI/F129970274/0707/070855/456352/118.150.47.40', VPN name 'sinopac', peer host '210.59.255.161:80' address 'IP 210.59.255.161', connection 'tcp_TxRx' local address 'IP | Event: Session reconnecting
Response Code: 0 | Event Code: 13 | Info: host '210.59.255.161:80', hostname '210.59.255.161:80' IP 210.59.255.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total reconnection attempt 1 of 10) | Event: Session reconnected


SDK NOTICE Mon Jul 08 00:27:34.561 2024 solClient.c:11120                    (31351f000) Session '(c2,s1)_sinopac' keep-alive on tcp_TxRx detected session down, client name 'PYAPI/F129970274/0707/134447/567859/118.150.47.40', VPN name 'sinopac', peer host '210.59.255.161:80' address 'IP 210.59.255.161', connection 'tcp_TxRx' local address 'IP 192.168.100.2:59350'


Response Code: 0 | Event Code: 12 | Info: Session '(c2,s1)_sinopac' keep-alive on tcp_TxRx detected session down, client name 'PYAPI/F129970274/0707/134447/567859/118.150.47.40', VPN name 'sinopac', peer host '210.59.255.161:80' address 'IP 210.59.255.161', connection 'tcp_TxRx' local address 'IP | Event: Session reconnecting
Response Code: 0 | Event Code: 13 | Info: host '210.59.255.161:80', hostname '210.59.255.161:80' IP 210.59.255.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total reconnection attempt 1 of 10) | Event: Session reconnected


SDK NOTICE Mon Jul 08 00:27:34.772 2024 solClient.c:11120                    (30442b000) Session '(c1,s1)_sinopac' keep-alive on tcp_TxRx detected session down, client name 'PYAPI/F129970274/0707/075254/526873/118.150.47.40', VPN name 'sinopac', peer host '210.59.255.161:80' address 'IP 210.59.255.161', connection 'tcp_TxRx' local address 'IP 192.168.100.2:59349'
SDK NOTICE Mon Jul 08 00:27:34.779 2024 solClient.c:11120                    (17357b000) Session '(c0,s1)_sinopac' keep-alive on tcp_TxRx detected session down, client name 'PYAPI/F129970274/0707/070855/456352/118.150.47.40', VPN name 'sinopac', peer host '210.59.255.161:80' address 'IP 210.59.255.161', connection 'tcp_TxRx' local address 'IP 192.168.100.2:59352'


Response Code: 0 | Event Code: 12 | Info: Session '(c1,s1)_sinopac' keep-alive on tcp_TxRx detected session down, client name 'PYAPI/F129970274/0707/075254/526873/118.150.47.40', VPN name 'sinopac', peer host '210.59.255.161:80' address 'IP 210.59.255.161', connection 'tcp_TxRx' local address 'IP | Event: Session reconnecting
Response Code: 0 | Event Code: 12 | Info: Session '(c0,s1)_sinopac' keep-alive on tcp_TxRx detected session down, client name 'PYAPI/F129970274/0707/070855/456352/118.150.47.40', VPN name 'sinopac', peer host '210.59.255.161:80' address 'IP 210.59.255.161', connection 'tcp_TxRx' local address 'IP | Event: Session reconnecting
Response Code: 0 | Event Code: 13 | Info: host '210.59.255.161:80', hostname '210.59.255.161:80' IP 210.59.255.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total reconnection attempt 1 of 10) | Event: Session reconnected
Response Code: 0 | Event Code: 13 | Info: host '210.59.255.161:80', hostname '210.59.255.161:80' IP 210.59.255.16